In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
import importlib
import configparser
import pymysql

In [ ]:
base_url = 'https://pokemongo.inven.co.kr/dataninfo/pokemon/'
base_soup = BeautifulSoup(requests.get(base_url).text, 'html.parser')
detail_urls = ['https://pokemongo.inven.co.kr'+a['href'] for a in base_soup.select('#pokemongoList > ul > li > a')]

In [59]:
iter_num = 0
poke_info_dic = {}
poke_info_element_list_for_db = [
        '포케몬넘버', '포케몬이름', '이미지url', '타입1', '타입2', '단일타입', '진화사탕', '진화비용', '최대 CP', '공격력', '방어력', 
        '체력', '포획 확률', '노말', '격투', '비행', '독', '땅', '바위', '벌레', '고스트', '강철', '불꽃', '물', '풀', '전기', 
        '에스퍼', '얼음', '드래곤', '악', '페어리', '진화단계', '최종진화']
poke_type_match_element_list = ['노말', '격투', '비행', '독', '땅', '바위', '벌레', '고스트', '강철', '불꽃', '물', '풀', '전기', '에스퍼', '얼음', '드래곤', '악', '페어리']

for each_url in detail_urls:
    detail_soup = BeautifulSoup(requests.get(each_url).text,'html.parser')
    # detail_soup
    poke_info = {}
    
    title_text = detail_soup.select_one('#articleView > div.articleContent > h1').get_text()
    poke_info['포케몬넘버'] = title_text.split('.')[0].strip()
    poke_info['포케몬이름'] = title_text.split('.')[1].strip()
    
    poke_info['이미지url'] = 'http:'+detail_soup.select_one('#articleView > div.articleContent > div.pokemonimage > img')['src']
    poke_type_text = detail_soup.select_one('#articleView > div.articleContent > div.pokemontype > ul > li.poekmontype').get_text()
    poke_info['타입1'] = poke_type_text.split()[0].strip()
    try:
        poke_info['타입2'] = poke_type_text.split()[1].strip()
        poke_info['단일타입'] = 'n'
    except:
        poke_info['타입2'] = ''
        poke_info['단일타입'] = 'y'
    try:
        poke_info['진화사탕'] = 'http:'+detail_soup.select_one('#articleView > div.articleContent > div.pokemontype > ul > li.last.evolvecandy > img')['src']
        poke_info['진화비용'] = detail_soup.select_one('#articleView > div.articleContent > div.pokemontype > ul > li.last.evolvecandy').get_text()        
    except:
        poke_info['진화사탕'] = ''
        poke_info['진화비용'] = '0'
    stat_soup = detail_soup.select_one('#articleView > div.articleContent > div.pokemontype > ul > li:nth-child(6)')
    poke_info['최대 CP'] = stat_soup.get_text()
    stat_soup = stat_soup.find_next_sibling().find_next_sibling()
    poke_info['공격력'] = stat_soup.get_text()
    stat_soup = stat_soup.find_next_sibling().find_next_sibling()
    poke_info['방어력'] = stat_soup.get_text()
    stat_soup = stat_soup.find_next_sibling().find_next_sibling()
    poke_info['체력'] = stat_soup.get_text()
    stat_soup = stat_soup.find_next_sibling().find_next_sibling()
    poke_info['포획 확률'] = stat_soup.get_text()
    
    match_soup = detail_soup.select_one('#articleView > div.articleContent > div.wrapContent > div.typeData > table > thead')
    th_texts = [th.get_text() for th in match_soup.find_all('th')]
    td_texts = [td.get_text() for td in match_soup.find_all('td')]
    poke_info['상성'] = dict(zip(th_texts, td_texts))
    
    evol_soup = detail_soup.select_one('#articleView > div.articleContent > div.groupList > ul')
    evol_step = 1
    evol_name_list = []
    evol_step_list = []
    for li in evol_soup.find_all('li'):
        if li.has_attr('class'):
            # arrow
            evol_step += 1
        else:
            # pokemon
            temp_name = li.select_one('span > a').get_text().split('.')[1].strip()
            if poke_info['포케몬이름'] == temp_name:
                poke_info['진화단계'] = evol_step
            evol_name_list.append(temp_name)
            evol_step_list.append(evol_step)
    poke_info['진화'] = list(zip(evol_step_list, evol_name_list))
    if poke_info['진화단계'] == str(evol_step):
        poke_info['최종진화'] = 'y'
    else:
        poke_info['최종진화'] = 'n'
    
    poke_info_dic[poke_info['포케몬이름']] = poke_info
    
    iter_num += 1
    # if iter_num > 5:
    #     break
    time.sleep(0.5)

In [83]:
len(poke_info_dic.keys())
poke_info_dic['이브이']['상성'].keys()

712

dict_keys(['노말', '격투', '비행', '독', '땅', '바위', '벌레', '고스트', '강철', '불꽃', '물', '풀', '전기', '에스퍼', '얼음', '드래곤', '악', '페어리'])

In [120]:
import os
import pathlib

conf_path = os.path.dirname(pathlib.Path.cwd()) + f'\ignore\config.ini'
conf = configparser.ConfigParser()
conf.read(conf_path)
db_conf = dict(conf['POKEMON_DB_CONFIG'])
db_conf['port'] = int(db_conf['port'])
conx = pymysql.connect(**db_conf)
cur = conx.cursor()

['c:\\Users\\jchoi\\Coding\\python\\ignore\\config.ini']

In [87]:
poke_info_element_list_for_db = [
        '포케몬넘버', '포케몬이름', '이미지url', '타입1', '타입2', '단일타입', '진화사탕', '진화비용', '최대 CP', '공격력', '방어력', 
        '체력', '포획 확률', '노말', '격투', '비행', '독', '땅', '바위', '벌레', '고스트', '강철', '불꽃', '물', '풀', '전기', 
        '에스퍼', '얼음', '드래곤', '악', '페어리', '진화단계', '최종진화']

In [ ]:
a = ['포케몬넘버', '포케몬이름']
b = ['3', '재미니']
insert_into_tb(cur, 'pokemon_info_tb', a, b)

In [132]:
for poke in poke_info_dic.keys():
    temp_data = (poke_info_dic[poke])
    temp_data.update(temp_data['상성'])
    poke_data = [str(temp_data[e]) for e in poke_info_element_list_for_db]
    insert_into_tb(cur, 'pokemon_info_tb', poke_info_element_list_for_db, poke_data)

insert into pokemon_info_tb (`포케몬넘버`,`포케몬이름`,`이미지url`,`타입1`,`타입2`,`단일타입`,`진화사탕`,`진화비용`,`최대 CP`,`공격력`,`방어력`,`체력`,`포획 확률`,`노말`,`격투`,`비행`,`독`,`땅`,`바위`,`벌레`,`고스트`,`강철`,`불꽃`,`물`,`풀`,`전기`,`에스퍼`,`얼음`,`드래곤`,`악`,`페어리`,`진화단계`,`최종진화`)
    values('1','이상해씨','http://static.inven.co.kr/image_2011/site_image/pokemongo/pokemonimage/pokemon_001.png?v=20181121a','풀','독','n','http://static.inven.co.kr/image_2011/pokemongo/dataninfo/pokemon/evolvecandy_20x20.png','25','1115','118','111','128','20%',' - ','0.625','1.6',' - ',' - ',' - ',' - ',' - ',' - ','1.6','0.625','0.391','0.625','1.6','1.6',' - ',' - ','0.625','1','n');


DataError: (1366, "Incorrect double value: ' - ' for column `pokemon`.`pokemon_info_tb`.`노말` at row 1")

In [123]:
def insert_into_tb(cur: pymysql.Connection.cursor, table_name: str, insert_columns: list, data_list: list):
    wrapped_insert_columns = ['`'+v+'`' for v in insert_columns]
    wrapped_data_list = ['\''+v+'\'' for v in data_list]
    sql = f"""insert into {table_name} ({','.join(wrapped_insert_columns)})
    values({','.join(wrapped_data_list)});"""
    print(sql)
    cur.execute(sql)

In [ ]:
cur = conx.cursor()
cur.execute('show tables')

In [124]:
conx.commit()

In [68]:
insert_into_tb(None, "pokemon_tb", ["a", "b"], ["1", "2"])

insert into pokemon_tb (a,b)
    values(1,2);


In [117]:
conx.close()